# User study questionnaire  
Mock and plots

Reference: KMunicate - https://bmjopen.bmj.com/content/9/9/e030215


In [ ]:
# TODO: generate 7 features: X0, XA, XA, XAY, XAY, XY, XY


In [ ]:
# TODO: generate results

## Main part
 * how much do you agree with the problem (Setting/setup)
 * how much do you agree with the need for a solution
 * how much did the visualization improve your interpretation. 
 [also allow for additional free-text comments on each]

### Participant details

* Principle role / background (school of causality)
* experience in causality years
* familiarity with Matching / IPW
* experience in reading/interpreting Love plots [years]   ?   # really break it down to reading/generating?
* experience in generating Love plots [years] ?
* continent of primary work


## Ablation
Compare against standard Love plot:
 * size
 * opacity
 * order
 * (size, opacity)
 * (size, opacity, order)
 [score each 0-10] [‘Less useful’, ‘Equal/no preference’, ‘A bit more useful’, ‘Somewhat more useful’ and ‘Much more useful’]
 [Then rank]
 [free text comments]